In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from IPython.display import display_html

!pip install lxml

!pip install html5lib

import lxml

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install folium
import folium

!pip install bs4
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


print("Libraries imported.")

Libraries imported.


## Extracting data from Wikipedia and cleaning it

In [2]:
Wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
Wiki.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
Wiki=Wiki[Wiki.Borough != 'Not assigned']

In [5]:
Wiki.loc[Wiki['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = Wiki['Borough']

## Getting the latitude and longitude coordenates

In [6]:
location = pd.read_csv('https://cocl.us/Geospatial_data')

In [7]:
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df = pd.merge(Wiki, location, on='Postal Code', how='left')

In [9]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Exploring and clustering the neighborhoods in Toronto

In [10]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [12]:
for Latitude, Longitude, Borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    
    label = folium.Popup(parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [13]:
borough_names = list(df.Borough.unique())

In [14]:
borough_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_toronto.append(x)
        
borough_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [15]:
df = df[df['Borough'].isin(borough_toronto)].reset_index(drop=True)
print(df.shape)
df.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [16]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)


for Latitude, Longitude, Postal_Code, Borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Postal Code'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(Borough, Postal_Code, Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define 4square Credentials and Version

In [17]:
CLIENT_ID = 'S55XCC5SAKMVZIRQ1GDA2BB5T0ZYWAI4JJMVIUCYWPGAR13O' # your Foursquare ID
CLIENT_SECRET = 'IUAZQFZWTMRHEUEY0BVFYG3NXALY141MFSUFD5H3ZMJJIDNW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S55XCC5SAKMVZIRQ1GDA2BB5T0ZYWAI4JJMVIUCYWPGAR13O
CLIENT_SECRET:IUAZQFZWTMRHEUEY0BVFYG3NXALY141MFSUFD5H3ZMJJIDNW


## Searching for venues

In [18]:
search_query = 'coffee'
radius = 500
print(search_query + ' .... OK!')

coffee .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S55XCC5SAKMVZIRQ1GDA2BB5T0ZYWAI4JJMVIUCYWPGAR13O&client_secret=IUAZQFZWTMRHEUEY0BVFYG3NXALY141MFSUFD5H3ZMJJIDNW&ll=43.6534817,-79.3839347&v=20180605&query=coffee&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa3e95f251404486106d0fa'},
 'response': {'venues': [{'id': '59f784dd28122f14f9d5d63d',
    'name': 'HotBlack Coffee',
    'location': {'address': '245 Queen Street West',
     'crossStreet': 'at St Patrick St',
     'lat': 43.65036434800487,
     'lng': -79.38866907575726,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65036434800487,
       'lng': -79.38866907575726}],
     'distance': 515,
     'postalCode': 'M5V 1Z4',
     'cc': 'CA',
     'neighborhood': 'Entertainment District',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['245 Queen Street West (at St Patrick St)',
      'Toronto ON M5V 1Z4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'pr

## Geting relevant part of JSON and transforming it into a pandas dataframe

In [21]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,59f784dd28122f14f9d5d63d,HotBlack Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1604577631,False,245 Queen Street West,at St Patrick St,43.650364,-79.388669,"[{'label': 'display', 'lat': 43.65036434800487...",515,M5V 1Z4,CA,Entertainment District,Toronto,ON,Canada,"[245 Queen Street West (at St Patrick St), Tor...",463001529
1,4b0aaa8ef964a520272623e3,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1604577631,False,"483 Bay St, Bell Trinity Square",Bell Trinity Square,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",130,M5G 2C9,CA,NaN,Toronto,ON,Canada,"[483 Bay St, Bell Trinity Square (Bell Trinity...",NaN
2,4b44fc77f964a520cc0026e3,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1604577631,False,427 University Avenue,NaN,43.654053,-79.388090,"[{'label': 'display', 'lat': 43.65405317976302...",340,NaN,CA,NaN,Toronto,ON,Canada,"[427 University Avenue, Toronto ON, Canada]",NaN
3,4baa9f6cf964a520817a3ae3,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1604577631,False,401 Bay St.,at Richmond St. W,43.652135,-79.381172,"[{'label': 'display', 'lat': 43.65213455850074...",268,M5H 2Y4,CA,NaN,Toronto,ON,Canada,"[401 Bay St. (at Richmond St. W), Toronto ON M...",NaN
4,53e8acc4498ee294fb100183,Timothy's World Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1604577631,False,425 University Ave,Dundas,43.654270,-79.387448,"[{'label': 'display', 'lat': 43.65427, 'lng': ...",296,M5G 1T6,CA,NaN,Toronto,ON,Canada,"[425 University Ave (Dundas), Toronto ON M5G 1...",NaN


In [22]:
coffee_lat=[]
coffee_lon=[]
coffee_name=[]

for i in range(len(venues)):
    coffee_lat.append(venues[i]['location']['lat'])
    coffee_lon.append(venues[i]['location']['lng'])
    coffee_name.append(venues[i]['name'])
    
data={'name':coffee_name,'latitude':coffee_lat,'longitude':coffee_lon}
coffee=pd.DataFrame(data)
coffee.head()

,name,latitude,longitude
0,HotBlack Coffee,43.650364,-79.388669
1,Timothy's World Coffee,43.653436,-79.382314
2,Timothy's World Coffee,43.654053,-79.388090
3,Timothy's World Coffee,43.652135,-79.381172
4,Timothy's World Coffee,43.654270,-79.387448


In [23]:
coffee.groupby(['name']).count()

,latitude,longitude
name,,
Bluestone Lane Queen Station Coffee Shop,1,1
Bulldog Coffee,1,1
Coffee Antidote 9th Floor,1,1
Coffee Court,1,1
Coffee Exchange,1,1
Coffee office,1,1
Cozy Coffee,1,1
Dineen Coffee,1,1
Dutton Employment Law,1,1


## Clustering

The idea is to see where is the least dense area at the map. It could be a good place to open new coffee shop.

In [24]:
map_lat=coffee['latitude'][0]
map_lon=coffee['longitude'][0]
toronto_map=folium.Map(location=[map_lat,map_lon],zoom_start=13)
km=KMeans(n_clusters=6, random_state=0).fit(coffee[['latitude','longitude']])
x=np.arange(6)
ys = [i + x + (i*x)**2 for i in range(6)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
coffee.insert(0,"cluster",km.labels_)

for lat,long,name,cluster in zip(coffee['latitude'],coffee['longitude'],coffee['name'],coffee['cluster']):
    label='{}'.format(name)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)        
    

toronto_map